# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [3]:
import sys
# sys.path.append('../')
import numpy as np
import pandas as pd
import sklearn as sk

In [4]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [6]:
tokenized_documents = tokenizer(X)

Number of tokens:  23767229
Number of sentences:  2832806


In [7]:
X, vect = vectorizer(tokenized_documents)

/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
# most frequent words
print("Top 10 most frequent words in the dataset")
print(vect.get_feature_names_out()[:10])

# least frequent words
print("Top 10 least frequent words in the dataset")
print(vect.get_feature_names_out()[-10:])

Top 10 most frequent words in the dataset
['0' '00' '000' '0000' '000001' '00001' '000013' '0000soo' '0001'
 '000111052']
Top 10 least frequent words in the dataset
['¾' 'â' 'çay' 'çaykur' 'çelem' 'être' 'île' 'ît' 'ø' 'þ']


In [9]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

(454763, 120144) (113691, 120144) (454763,) (113691,)
  (0, 94340)	0.2698393204452672
  (0, 107461)	0.356573063664095
  (0, 60888)	0.6837599652000029
  (0, 31216)	0.37896669799521565
  (0, 25542)	0.19195720719400375
  (0, 79539)	0.28524356089653274
  (0, 69704)	0.22960104461763894
  (0, 66863)	0.13405538070127823


# Recurrent Neural Network
### Model starts here

In [11]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
import datetime
import os

2024-05-03 13:50:16.088485: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 13:50:16.137031: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 13:50:17.350613: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
def batch_generator(X, y, batch_size=32, num_classes=None, timesteps=1):
    num_samples = X.shape[0]
    features_per_timestep = X.shape[1] // timesteps
    while True:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = X[start:end].toarray()  # Convert only this slice to dense
            X_batch = X_batch.reshape(-1, timesteps, features_per_timestep)  # Reshape for LSTM
            y_batch = to_categorical(y[start:end], num_classes=num_classes)
            yield (X_batch, y_batch)


# Setup the model
model = Sequential([
    LSTM(256, input_shape=(None, X_train.shape[1]), activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(128, activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(np.max(y_train) + 1, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model using a generator
train_generator = batch_generator(X_train, y_train, batch_size=128, num_classes=np.max(y_train) + 1, timesteps=1)
model.fit(train_generator, epochs=20, steps_per_epoch=int(np.ceil(X_train.shape[0] / 128)))

2024-05-03 13:50:18.047585: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13290 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:ca:00.0, compute capability: 8.6
/opt/mamba/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20


I0000 00:00:1714744221.648736   26245 service.cc:145] XLA service 0x561efc308130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714744221.648803   26245 service.cc:153]   StreamExecutor device (0): NVIDIA A2, Compute Capability 8.6
2024-05-03 13:50:21.757225: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-03 13:50:22.302072: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900
I0000 00:00:1714744226.594477   26419 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 268 bytes spill stores, 260 bytes spill loads

I0000 00:00:1714744226.704992   26409 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_60', 64 bytes spill stores, 64 bytes spill loads

I0000 00:00:1714744226.928

   2/3553 ━━━━━━━━━━━━━━━━━━━━ 4:32 77ms/step - accuracy: 0.3633 - loss: 1.7895   

I0000 00:00:1714744233.761103   26245 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_multiply_select_fusion_3', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1714744233.777667   26245 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3552/3553 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.7014 - loss: 0.8369

I0000 00:00:1714744492.093305   27139 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1714744492.524821   27183 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1714744492.578977   27158 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 252 bytes spill stores, 172 bytes spill loads

I0000 00:00:1714744492.674624   27184 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_48', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1714744492.689486   27127 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_58', 24 bytes spill stores, 20 bytes spill loads

I0000 00:00:1714744492.811445   27223 asm_

3553/3553 ━━━━━━━━━━━━━━━━━━━━ 281s 75ms/step - accuracy: 0.7014 - loss: 0.8369
Epoch 2/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 257s 72ms/step - accuracy: 0.7977 - loss: 0.5606
Epoch 3/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 262s 74ms/step - accuracy: 0.8674 - loss: 0.3773
Epoch 4/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 257s 72ms/step - accuracy: 0.9096 - loss: 0.2581
Epoch 5/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 257s 72ms/step - accuracy: 0.9340 - loss: 0.1882
Epoch 6/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 259s 73ms/step - accuracy: 0.9503 - loss: 0.1422
Epoch 7/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 259s 73ms/step - accuracy: 0.9611 - loss: 0.1114
Epoch 8/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 260s 73ms/step - accuracy: 0.9680 - loss: 0.0919
Epoch 9/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 258s 73ms/step - accuracy: 0.9736 - loss: 0.0758
Epoch 10/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 260s 73ms/step - accuracy: 0.9773 - loss: 0.0659
Epoch 11/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 258s 73ms/step - accuracy: 0.9807 - loss: 0.0565
Epoch 12

In [13]:
test_generator = batch_generator(X_test, y_test, batch_size=32, num_classes=np.max(y_train) + 1, timesteps=1)

In [14]:
# Evaluate the model
predictions_prob = model.predict(test_generator, steps=int(np.ceil(X_test.shape[0] / 32)))
predictions = np.argmax(predictions_prob, axis=1)

# Metrics
print(metrics.confusion_matrix(y_test, predictions))
print(metrics.classification_report(y_test, predictions))

I0000 00:00:1714749373.317353   35709 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_27', 16 bytes spill stores, 16 bytes spill loads



3550/3553 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

I0000 00:00:1714749412.643711   35984 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_27', 16 bytes spill stores, 16 bytes spill loads



3553/3553 ━━━━━━━━━━━━━━━━━━━━ 42s 11ms/step
[[ 7119  1328   527   188  1164]
 [  734  3259   680   295   887]
 [  403   842  4502   949  1789]
 [  195   405   974  8645  5904]
 [  516   708   974  2977 67727]]
              precision    recall  f1-score   support

           1       0.79      0.69      0.74     10326
           2       0.50      0.56      0.53      5855
           3       0.59      0.53      0.56      8485
           4       0.66      0.54      0.59     16123
           5       0.87      0.93      0.90     72902

    accuracy                           0.80    113691
   macro avg       0.68      0.65      0.66    113691
weighted avg       0.80      0.80      0.80    113691



In [15]:
# # Save the model with current date and time in model folder

# Create a folder named _models in the current directory
if not os.path.exists('_models'):
    os.makedirs('_models')
    
model.save(f'_models/config1_rnn_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')

In [17]:
# test the model
test_sentences = [
    'This is a good product',
    'This is a bad product',
    'This is a product',
    'This is a very good product',
    'This is a very bad product',
    'That was bad'
]

for sentence in test_sentences:
    test_tokenized = tokenizer([sentence])
    test_vec = vect.transform(test_tokenized)
    test_vec_dense = test_vec.toarray()  # Convert sparse tensor to dense tensor
    test_vec_lstm = test_vec_dense.reshape(-1, 1, X.shape[1])
    result = model.predict(test_vec_lstm)
    predicted_class = np.argmax(result, axis=-1)
    predicted_score = predicted_class
    print(f"Test sentence: {sentence}")
    print(f"Predicted score: {predicted_score}")
    print()

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


Test sentence: This is a good product
Predicted score: [5]

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Test sentence: This is a bad product
Predicted score: [5]

Number of tokens:  1
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Test sentence: This is a product
Predicted score: [4]

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Test sentence: This is a very good product
Predicted score: [5]

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Test sentence: This is a very bad product
Predicted score: [5]

Number of tokens:  1
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Test sentence: That was bad
Predicted score: [5]

